In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        'streamlit',
        'pandas',
        'numpy',
        'plotly',
        'textblob',
        'vaderSentiment',
        'wordcloud',
        'matplotlib',
        'seaborn'
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✅ {package} installed")
        except subprocess.CalledProcessError:
            print(f"❌ Failed to install {package}")

install_packages()

In [ ]:
import tweepy
import pandas as pd
from datetime import datetime
import os
import json
import time
from typing import List, Dict

class TwitterClient:
    def __init__(self):
        self.bearer_token = os.getenv('TWITTER_BEARER_TOKEN')
        self.api_key = os.getenv('TWITTER_API_KEY')
        self.api_secret = os.getenv('TWITTER_API_SECRET')
        self.access_token = os.getenv('TWITTER_ACCESS_TOKEN')
        self.access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        
        if not self.bearer_token:
            print("Twitter API credentials not found. Using simulated data.")
            self.client = None
        else:
            try:
                self.client = tweepy.Client(
                    bearer_token = self.bearer_token,
                    api_key = self.api_key,
                    api_secret = self.api_secret,
                    access_token = self.access_token,
                    access_token_secret = self.access_token_secret,
                    wait_on_rate_limit=True
                )
                print("✅ Twitter API client initialized successfully")
            except Exception as e:
                print(f"❌ Error initialized Twitter client: {e}")
                self.client = None
                
    def search_tweets(self, query: str, max_results: int = 10) -> List[Dict]:
        if not self.client:
            return self._get_simulated_tweets(max_results)
        
        try:
            tweets = self.client.searcg_recent_tweets(
                query=query,
                max_results=max_results,
                tweets_fields=['created_at', 'author_id', 'public_metrics', 'context_annotations']
            )
            
            if not tweets.data:
                return []
            
            tweet_list = []
            for tweet in tweets.data:
                tweet_data = {
                    'text': tweet.text,
                    'timestamp': tweet.created_at,
                    'user': f"user_{tweet.author_id}",
                    'likes': tweet.public_metrics['like_count'] if tweet.public_metrics else 0,
                    'retweets': tweet.public_metrics['retweet_count'] if tweet.public_metrics else 0,
                    'tweet_id': tweet.id
                }
                tweet_list.append(tweet_data)
                
            return tweet_list
        
        except Exception as e:
            print(f"Error fetching tweets: {e}")
            return self._get_simulated_tweets(max_results)
        
    def _get_simulated_tweets(self, count: int) -> List[Dict]:
        from datetime import datetime, timedelta
        import random
        
        sample_tweets = [
            "Just discovered this amazing new coffee shop! ☕ #coffee #love",
            "Traffic is terrible today! 😤 #frustrated #commute",
            "Beautiful sunset tonight 🌅 #nature #peaceful",
            "My flight got delayed again... 😞 #travel #delays",
            "Excited for the weekend! 🎉 #happy #weekend",
            "This movie is absolutely incredible! Must watch 🍿 #movies",
            "Worst customer service ever! Very disappointed 😠 #complaint",
            "Learning Python is so rewarding! 💻 #coding #tech",
            "Rain ruined my picnic plans ☔ #weather #sad",
            "Just finished a great workout! 💪 #fitness #health"
        ]
        
        tweets = []
        for _ in range(count):
            tweet = {
                'text': random.choice(sample_tweets),
                'timestamp': datetime.now() - timedelta(seconds=random.randint(0, 3600)),
                'user': f"user_{random.randint(1000, 9999)}",
                'likes': random.randint(0, 100),
                'retweets': random.randint(0, 50),
                'tweet_id': f"sim_{random.randint(100000, 999999)}"
            }
            tweets.append(tweet)
        
        return tweets


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import random
import re
from collections import Counter
from wordcloud import WordCloud
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

print("✅ Libraries Imported Successfully")

In [ ]:
class TweetGenerator:
    """Generate sample teets"""
    
    def __init__(self):
        self.positive_tweets = [
            "I love this new product! Amazing quality! 😍",
            "Having a great day with friends! #blessed",
            "Just got promoted at work! So excited! 🎉",
            "Beautiful sunset today! Nature is amazing 🌅",
            "Great customer service experience! Highly recommend!",
            "Finally finished my project! Feeling accomplished ✨",
            "Best movie I've watched this year! Must see! 🍿",
            "Delicious food at this restaurant! Coming back soon!",
            "Love spending time with family! ❤️",
            "Successfully completed my workout! Feeling strong 💪"
        ]
        
        self.negative_tweets = [
            "Terrible customer service! Very disappointed 😠",
            "This product broke after one day! Waste of money!",
            "Stuck in traffic for 2 hours! Worst day ever!",
            "Can't believe how rude people can be! 😤",
            "My order was completely wrong! Very frustrated!",
            "Website is so slow and buggy! Fix this please!",
            "Overpriced and poor quality! Not worth it!",
            "Rain ruined my outdoor plans! So annoying ☔",
            "Long wait times everywhere! What's happening?",
            "Technology failing me when I need it most! 📱💔"
        ]
        
        self.neutral_tweets = [
            "Just finished reading a book about data science.",
            "Weather forecast shows rain tomorrow.",
            "The meeting is scheduled for 3 PM.",
            "Coffee shop on main street opens at 7 AM.",
            "Train arrives at platform 2 in 5 minutes.",
            "New policy update will be effective next month.",
            "Concert tickets go on sale this Friday.",
            "Library hours: Monday to Friday 9 AM to 8 PM.",
            "Parking meter accepts both coins and cards.",
            "Store inventory will be updated weekly."
        ]
        
        self.topics = ["#technology", "#food", "#travel", "#work", "#family", "#health", "#sports", "#entertainment"]
        
    def generate_tweet(self):
        sentiment_type = random.choices(
            ['positive', 'negative', 'neutral'],
            weights = [0.4, 0.3, 0.3]
        )[0]
        
        if sentiment_type == 'positive':
            text = random.choice(self.positive_tweets)
        elif sentiment_type == 'negative':
            text = random.choice(self.negative_tweets)
        else:
            text = random.choice(self.neutral_tweets)
            
        if random.random() > 0.5:
            text += " " + random.choice(self.topics)
            
        return {
            'text': text,
            'timestamp': datetime.now() - timedelta(seconds=random.randint(0, 3600)),
            'user': f"user_{random.randint(1000, 9999)}",
            'likes': random.randint(0, 100),
            'retweets': random.randint(0, 50)
        }
        
    def generate_batach_tweets(self, count=10):
        return [self.generate_tweet() for _ in range(count)]    

In [ ]:
class SentimentAnalyzer:
    """Multi-modal sentiment analysis"""
    
    def __init__(self) -> None:
        self.vader_analyzer = SentimentIntensityAnalyzer()
        
    def clean_text(self, text):
        # Removing URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Removing mentions and all also
        text = re.sub(r'@\w+|#', '', text)
        text = ' '.join(text.split())
        
        return text.strip()
    
    def analyze_textblob(self, text):
        clean_text = self.clean_text(text)
        blob = TextBlob(clean_text)
        
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        if polarity > 0.1:
            sentiment = 'Positive'
        elif polarity < -0.1:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'polarity': polarity,
            'subjectivity': subjectivity,
            'confidence': abs(polarity)
        }
        
    def analyze_vader(self, text):
        clean_text = self.clean_text(text)
        scores = self.vader_analyzer.polarity_scores(clean_text)
        
        if scores['compound'] >= 0.05:
            sentiment = 'Positive'
        elif scores['compound'] <= -0.05:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'positive': scores['pos'],
            'neutral': scores['neu'],
            'negative': scores['neg'],
            'compound': scores['compound']
        }
        
    def analyze_combined(self, text):
        textblob_result = self.analyze_textblob(text)
        vader_result = self.analyze_vader(text)
        
        if textblob_result['sentiment'] == vader_result['sentiment']:
            final_sentiment = textblob_result['sentiment']
            confidence = (abs(textblob_result['polarity']) + abs(vader_result['compound'])) / 2
        else:
            if abs(vader_result['compound']) > abs(textblob_result['polarity']):
                final_sentiment = vader_result['sentiment']
                confidence = abs(vader_result['compound'])
            else:
                final_sentiment = textblob_result['sentiment']
                confidence = abs(textblob_result['polarity'])
        
        return {
            'sentiment': final_sentiment,
            'confidence': confidence,
            'textblob': textblob_result,
            'vader': vader_result
        }
        
    def analyze_batch(self, texts):
        results = []
        
        for text in texts:
            result = self.analyze_combined(text)
            results.append(result)
    
        return results

In [ ]:
dashboard_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
from wordcloud import WordCloud
from collections import Counter
import io
import base64

def main():
    st.set_page_config(
        page_title="Real-Time Sentiment Analysis Dashboard",
        page_icon="📊",
        layout="wide"
    )
    
    st.title("📊 Real-Time Sentiment Analysis Dashboard")
    st.markdown("---")
    
    if 'tweet_generator' not in st.session_state:
        st.session_state.tweet_generator = TweetGenerator()
    if 'sentiment_analyzer' not in st.session_state:
        st.session_state.sentiment_analyzer = SentimentAnalyzer()
    if 'tweets_data' not in st.session_state:
        st.session_state.tweets_data = []
        
    st.sidebar.header("Controls")
    
    auto_refresh = st.sidebar.checkbox("Auto Refresh", value=False)
    refresh_interval = st.sidebar.slider("Refresh Interval (seconds)", 1, 10, 3)
    
    if st.sidebar.button("Generate New Tweets"):
        generate_new_tweets()
    
    if st.sidebar.button("Clear All Data"):
        st.session_state.tweets_data = []
        st.experimental_rerun()
        
    st.sidebar.header("Analyze Your Own Tweet")
    user_tweet = st.sidebar.text_area("Enter your tweet here:")
    if st.sidebar.button("Analyze") and user_tweet:
        analyze_user_tweet(user_tweet)
        
    col1, col2, col3 = st.columns(3)
    
    if not st.session_state.tweets_data:
        generate_new_tweets()
        
    display_metrics(col1, col2, col3)   
    
    display_charts()
    
    display_recent_tweets()
    
    if auto_refresh:
        time.sleep(refresh_interval)
        generate_new_tweets()
        st.experimental_rerun()
        
def generate_new_tweets():
    new_tweets = st.session_state.tweet_generator.generate_batach_tweets(5)
    
    for tweet in new_tweets:
        analysis = st.session_state.sentiment_analyzer.analyze_combined(tweet['text'])
        tweet_data = {
            'text': tweet['text'],
            'timestamp': tweet['timestamp'],
            'user': tweet['user'],
            'likes': tweet['likes'],
            'retweets': tweet['retweets'],
            'sentiment': analysis['sentiment'],
            'confidence': analysis['confidence'],
            'textblob_polarity': analysis['textblob']['polarity'],
            'vader_compound': analysis['vader']['compound']
        }
        
        st.session_state.tweets_data.append(tweet_data)
    
    if len(st.session_state.tweets_data) > 100:
        st.session_state.tweets_data = st.session_state.tweets_data[-100:]
    
def analyze_user_tweet(text):
    analysis = st.session_state.sentiment_analyzer.analyze_combined(text)
    
    st.sidebar.success(f"Sentiment: {analysis['sentiment']}")
    st.sidebar.info(f"Confidence: {analysis['confidence']:.2f}")
    
    tweet_data = {
        'text': text,
        'timestamp': datetime.now(),
        'user': 'You',
        'likes': 0,
        'retweets': 0,
        'sentiment': analysis['sentiment'],
        'confidence': analysis['confidence'],
        'textblob_polarity': analysis['textblob']['polarity'],
        'vader_compound': analysis['vader']['compound']
    }
    st.session_state.tweets_data.append(tweet_data)
    
def display_metrics(col1, col2, col3):
    if not st.session_state.tweets_data:
        return
    
    df = pd.DataFrame(st.session_state.tweets_data)
    
    total_tweets = len(df)
    col1.metric("Total Tweets", total_tweets)
    
    sentiment_counts = df['sentiment'].value_counts()
    positive_pct = (sentiment_counts.get('Positive', 0) / total_tweets) * 100
    col2.metric("Positive Sentiment", f"{positive_pct:.1f}%")
    
    avg_confidence = df['confidence'].mean()
    col3.metric("Avg Confidence", f"{avg_confidence:.2f}")
    
def display_charts():
    if not st.session_state.tweets_data:
        return
    
    df = pd.DataFrame(st.session_state.tweets_data)
    
    col1, col2 = st.columns(2)
    
    # Sentiment distribution pie chart
    with col1:
        st.subheader("Sentiment Distribution")
        sentiment_counts = df['sentiment'].value_counts()
        
        colors = {'Positive': '#2E8B57', 'Negative': '#DC143C', 'Neutral': '#4682B4'}
        fig = px.pie(
            values=sentiment_counts.values,
            names=sentiment_counts.index,
            title="Sentiment Distribution",
            color=sentiment_counts.index,
            color_discrete_map=colors
        )
        st.plotly_chart(fig, use_container_width=True)
    
    # Sentiment over time
    with col2:
        st.subheader("Sentiment Over Time")
        df_sorted = df.sort_values('timestamp')
        
        # Create color mapping
        color_map = {'Positive': 'green', 'Negative': 'red', 'Neutral': 'blue'}
        df_sorted['color'] = df_sorted['sentiment'].map(color_map)
        
        fig = px.scatter(
            df_sorted,
            x='timestamp',
            y='confidence',
            color='sentiment',
            title='Sentiment Confidence Over Time',
            color_discrete_map=colors
        )
        st.plotly_chart(fig, use_container_width=True)
        
    # Word cloud
    st.subheader("Wrod Cloud")
    all_text = ' '.join(df['text'].tolist())
    
    if all_text:
        wordcloud = WordCloud(
            width=800,
            height=400,
            background_color='white',
            colormap='viridis'
        ).generate(all_text)
        
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')
        st.pyplot(fig)
        
    st.subheader("COnfidence Distribution by Sentiment")
    fig = px.box(
        df,
        x='sentiment',
        y='confidence',
        color='sentiment',
        title='Confindence Scores by Sentiment',
        color_discrete_map=colors
    )
    
    st.plotly_chart(fig, use_container_width=True)
    
def display_recent_tweets():
    """Display recent tweets table"""
    if not st.session_state.tweets_data:
        return
    
    st.subheader("Recent Tweets")
    
    df = pd.DataFrame(st.session_state.tweets_data)
    df_display = df.sort_values('timestamp', ascending=False).head(10)
    
    # Format the display
    df_display['Time'] = df_display['timestamp'].dt.strftime('%H:%M:%S')
    df_display['Sentiment'] = df_display['sentiment']
    df_display['Confidence'] = df_display['confidence'].round(2)
    df_display['Tweet'] = df_display['text'].str[:100] + '...'
    
    # Color code sentiments
    def color_sentiment(val):
        if val == 'Positive':
            return 'background-color: #90EE90'
        elif val == 'Negative':
            return 'background-color: #FFB6C1'
        else:
            return 'background-color: #ADD8E6'
    
    styled_df = df_display[['Time', 'Tweet', 'Sentiment', 'Confidence']].style.applymap(
        color_sentiment, subset=['Sentiment']
    )
    
    st.dataframe(styled_df, use_container_width=True)

if __name__ == "__main__":
    main()
'''
    
with open('dashboard.py', 'w', encoding='utf-8') as f:
    f.write(dashboard_code)

print("✅ Dashboard file created: dashboard.py")

In [ ]:
def run_dashboard():
    """Run the Streamlit dashboard"""
    print("🚀 Starting Real-Time Sentiment Analysis Dashboard...")
    print("📊 The dashboard will open in your browser")
    print("---")
    
    # Run streamlit
    import subprocess
    subprocess.run([sys.executable, '-m', 'streamlit', 'run', 'dashboard.py'])
    
run_dashboard()
    